In [16]:
import os
from dotenv import load_dotenv, dotenv_values
import pandas as pd

In [17]:
#docu1.py
#PURPOSE: To save csv generated data from docusign to aws database.

In [23]:
#CHECK IF FILE "Envelope Recipient Report.csv" EXIST
def file_readiness_checker():
    #File "Envelope Recipient Report.csv" extracted from Docusign
    print("Checking if 'Envelope Recipient Report.csv' exist")
    if os.path.exists("Envelope Recipient Report.csv"):
        print("'Envelope Recipient Report.csv' exist.")
        print("Checking if update is available.")
        
        #Check if it's the right file by checking if all columns are available and is in order
        df = pd.read_csv("Envelope Recipient Report.csv")
        df.head()
        
        #Check if update is available by checking if there is changes in the "Completed On (Date)" column, if no, then the file has not update. Return error notice
        
        #If there is changes, then proceed to update the database
    else:
        print("'Envelope Recipient Report.csv' does not exist.")

In [24]:
file_readiness_checker()

Checking if 'Envelope Recipient Report.csv' exist
'Envelope Recipient Report.csv' exist.
Checking if update is available.
